In [1]:
experiment_name = "sentiment_exp"

In [2]:
initial_prompt = '''
You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. 
Think through your analysis step by step using chain of thought reasoning. 
After your analysis, respond with a STRIC JSON dictionary containing two keys: 
"chain_of_thought" (your step-by-step reasoning) and "classification" (1 for positive, 0 for negative).

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here"
    "classification": 0 or 1,
}
Ensure that "chain_of_thought" contains your detailed analysis, and "classification" is strictly 0 or 1
'''

In [3]:
# Output format prompt
output_format_prompt = '''
Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here"
    "classification": 0 or 1,
}
Ensure that "chain_of_thought" contains your detailed analysis, and "classification" is strictly 0 or 1
'''

In [4]:
fp_comments = ""
fn_comments = ""
tp_comments = ""
invalid_comments = ""
prompt_engineering_comments = ""
validation_comments = ""

In [5]:
# Define output schema
output_schema = {
    'key_to_extract': 'classification',
    'value_mapping': {'1': 1,'0': 0},
    'regex_pattern': r'"classification":\s*(\d)',
    #
    'chain_of_thought_key': 'chain_of_thought',  
    'chain_of_thought_regex': r'"chain_of_thought":\s*"(.*?)"',
    #
    'use_json_mode': True,
}

In [6]:
# Set number of optimization iterations
iterations = 2

In [7]:
# Define model providers and models for evaluation and optimization
# eval_provider = "ollama"
# eval_model = "llama3.1"
# optim_provider = "ollama"
# optim_model = "llama3.1"
eval_provider = "openai"
eval_model = "gpt-4o-mini"
optim_provider = "openai"
optim_model = "gpt-4o-mini"

In [8]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "sentiments.csv"
sample_size = 10

------------------------------------------------------------------------------------------

In [9]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(grandparent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [10]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['Text', 'Sentiment'])
eval_data.columns = ['text', 'label']
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=round(sample_size/2), random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (10, 2)
                                                text  label
0  Was this based on a comic-book? A video-game? ...      1
1  If you ask me the first one was really better ...      0
2  When I was a kid, I loved "Tiny Toons". I espe...      1
3  I hate guns and have never murdered anyone, bu...      0
4  I do not have much to say than this is a great...      1


/var/folders/cy/w_c5c0zs70lcg80xwzft9fjh0000gn/T/ipykernel_42759/3552288940.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=round(sample_size/2), random_state=42))


In [11]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt = initial_prompt,
    eval_data = eval_data,
    iterations = iterations,
    eval_provider = eval_provider,
    eval_model = eval_model,
    eval_temperature = 0.7,
    optim_provider = optim_provider,
    optim_model = optim_model,
    optim_temperature = 0,
    use_cache = True,
    output_format_prompt = output_format_prompt,
    output_schema = output_schema,
    fp_comments = fp_comments,
    fn_comments = fn_comments,
    tp_comments = tp_comments,
    invalid_comments = invalid_comments,
    prompt_engineering_comments = prompt_engineering_comments,
    validation_comments = validation_comments,
    experiment_name = experiment_name,
    skip_prompt_validation = True,
)


Detected problem type: binary
Selected evaluation provider: openai
Selected evaluation model: gpt-4o-mini
Evaluation temperature: 0.7
Selected optimization provider: openai
Selected optimization model: gpt-4o-mini
Optimization temperature: 0
Estimated token usage: 74080
Estimated cost: $0.09

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/2


╭───────────────────────────────────────────── Current Full Prompt ─────────────────────────────────────────────╮
│                                                                                                               │
│ You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment.  │
│ Think through your analysis step by step using chain of thought reasoning.                                    │
│ After your analysis, respond with a STRIC JSON dictionary containing two keys:                                │
│ "chain_of_thought" (your step-by-step reasoning) and "classification" (1 for positive, 0 for negative).       │
│                                                                                                               │
│ Provide your response as a JSON dictionary with the following structure:                                      │
│ {                                                                                                             │
│     "chain_of_thought": "Your step-by-step reasoning here"                                                    │
│     "classification": 0 or 1,                                                                                 │
│ }                                                                                                             │
│ Ensure that "chain_of_thought" contains your detailed analysis, and "classification" is strictly 0 or 1       │
│                                                                                                               │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

-----------------------------------
Processing text 1/10 .....
Using cached output for text 1/10
Prediction 1/10: 0 | Ground Truth: 1 ❌ (FN)
-----------------------------------
Processing text 2/10 .....
Using cached output for text 2/10
Prediction 2/10: 0 | Ground Truth: 0 ✅ (TN)
-----------------------------------
Processing text 3/10 .....
Using cached output for text 3/10
Prediction 3/10: 1 | Ground Truth: 1 ✅ (TP)
-----------------------------------
Processing text 4/10 .....
Using cached output for text 4/10
Prediction 4/10: 0 | Ground Truth: 0 ✅ (TN)
-----------------------------------
Processing text 5/10 .....
Using cached output for text 5/10
Prediction 5/10: 1 | Ground Truth: 1 ✅ (TP)
-----------------------------------
Processing text 6/10 .....
Using cached output for text 6/10
Prediction 6/10: 0 | Ground Truth: 0 ✅ (TN)
-----------------------------------
Processing text 7/10 .....
Using cached output for text 7/10
Prediction 7/10: 1 | Ground Truth: 1 ✅ (TP)
-------------

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 1.0000 │
│ Recall              │ 0.8000 │
│ Accuracy            │ 0.9000 │
│ F1-score            │ 0.8889 │
│                     │        │
├─────────────────────┼────────┤
│ Valid Predictions   │     10 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘

   Confusion Matrix    
┏━━━━━━━━━━━━━┳━━━┳━━━┓
┃ True↓/Pred→ ┃ 0 ┃ 1 ┃
┡━━━━━━━━━━━━━╇━━━╇━━━┩
│ 0           │ 5 │ 0 │
├─────────────┼───┼───┤
│ 1           │ 1 │ 4 │
└─────────────┴───┴───┘

...

You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. 
Think through your analysis step by step using chain of thought reasoning. 
After your analysis, respond with a STRIC JSON dictionary containing two keys: 
"chain_of_thought" (your step-by-step reasoning) and "classification" (1 for positive, 0 for negative).

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here"
    "classification": 0 or 1,
}
Ensure that "chain_of_thought" contains your detailed analysis, and "classification" is strictly 0 or 1

╭────────────────╮
│ Output Format: │
╰────────────────╯

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here"
    "classification": 0 or 1,
}
Ensure that "chain_of_thought" contains your detailed analysis, and "classification" is strictly 0 or 1

                                   Comparison of All Iterations                                   
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Valid Predictions ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    1.0000 │ 0.8000 │   0.9000 │   0.8889 │           10.0000 │              0.0000 │
│     2     │    0.6667 │ 0.8000 │   0.7000 │   0.7273 │           10.0000 │              0.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴───────────────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/examples/binary_classifier/runs/sentiment_exp_Wed_30-Oct-2024_16-18-07
